# <b>1 <span style='color:#2ae4f5'>|</span> Genoma Class </b>


In [2]:
using Random
using Graphs
using IterTools
using Printf
using Distributions

mutable struct Genoma
    genotipo::Vector{String}
    copyGenotipo::Vector{String}
    numberOfGenes::Int
    faultChance::Float64
    nInputs::Int
    nOutputs::Int
    possiblesOutputs::Int
    fitness::Float64
    noiseFitness::Float64
    rateMutation::Float64
    deadGenesRate::Float64
    activeGenesRate::Float64
    flagDeadActive::Int
    outputsIndexes::Vector{String}
    indexDeadGenes::Vector{Int}
    indexActiveGenes::Vector{Int}
    Stochasticity::Float64
    ToEvaluate::Vector{Bool}
    identified::Bool
end

function Genoma(; numberOfGenes=60, nInputs=4, nOutputs=1, rateMutation=0.10)
    list = []
    for i in 1:nOutputs
        push!(list,string(numberOfGenes+nInputs-i+1))
    end
    return Genoma([], [], numberOfGenes, 0.0, nInputs, nOutputs, 2^nInputs, 0.0, 0.0,
                  rateMutation, 0.6, 0.4, 0, list,[], [], 0.003, [],false)
end

function setFitness(genoma::Genoma, fitness::Float64)
    genoma.fitness = fitness
end

function setGenotipo(genoma::Genoma, a::String)
    # Divide a string pelos espaços
    pairs = split(a, " ")

    # Cria um vetor para armazenar os resultados
    updated_pairs = []

    for pair in pairs
        # Divide cada par pelo "-"
        numbers = split(pair, "-")
        # Converte para inteiro, soma 1 e converte de volta para string
        updated_pair = string(parse(Int, numbers[1]) + 1, "-", parse(Int, numbers[2]) + 1)
        # Adiciona o par atualizado à lista
        push!(updated_pairs, string(updated_pair))
    end    
    genoma.genotipo = copy(updated_pairs)
end

function setGenotipo(genoma::Genoma, a::Vector{String})
    genoma.genotipo = copy(a)
end

function getGenotipo(genoma::Genoma)
    join(genoma.genotipo, " ")
end

function setFaultChance(genoma::Genoma, newFaultChance::Float64=0.0)
    genoma.faultChance = newFaultChance
end

function setStochasticity(genoma::Genoma, newStochasticity::Float64)
    genoma.Stochasticity = newStochasticity
end

function setRateMutation(genoma::Genoma, newRateMutation::Float64)
    genoma.rateMutation = newRateMutation
end

function setNumberOfGenes(genoma::Genoma, newLength::Int)
    genoma.numberOfGenes = newLength
end

function copyGene(genomaSource::Genoma, genomaDestiny::Genoma)
    genomaDestiny.genotipo = copy(genomaSource.genotipo)
    genomaDestiny.copyGenotipo = copy(genomaSource.copyGenotipo)
    genomaDestiny.fitness = genomaSource.fitness
    genomaDestiny.noiseFitness = genomaSource.noiseFitness
    genomaDestiny.flagDeadActive = genomaSource.flagDeadActive
    genomaDestiny.ToEvaluate = genomaSource.ToEvaluate
    genomaDestiny.possiblesOutputs = genomaSource.possiblesOutputs
    genomaDestiny.outputsIndexes = genomaSource.outputsIndexes
    genomaDestiny.identified = genomaSource.identified
end

function fill_Initial_Genome(genoma::Genoma)
    genoma.genotipo = ["" for _ in 1:genoma.numberOfGenes]
end

function generate_parent(genoma::Genoma)
    fill_Initial_Genome(genoma)
    for i in 1:genoma.numberOfGenes
        in1 = rand(1:i + genoma.nInputs - 1)
        in2 = rand(1:i + genoma.nInputs - 1)
        sin1 = string(in1)
        sin2 = string(in2)
        genoma.genotipo[i] = "$sin1-$sin2"
    end
end

function identify_deadGenes(genoma::Genoma)
    genoma.ToEvaluate = falses(genoma.numberOfGenes)  # Inicializa como falso
    
    for i in 1:genoma.nOutputs
        genoma.ToEvaluate[end - i + 1] = true  # Corrigido  
    end

    p = genoma.numberOfGenes
    while p >= 1
        if genoma.ToEvaluate[p]  # Corrigido
            inputs = split(genoma.genotipo[p], "-")
            input1 = parse(Int, inputs[1])
            input2 = parse(Int, inputs[2])
            x = input1 - genoma.nInputs
            y = input2 - genoma.nInputs
            if x >= 1
                genoma.ToEvaluate[x] = true  # Corrigido
            end
            if y >= 1
                genoma.ToEvaluate[y] = true  # Corrigido
            end
        end
        p -= 1
    end

end

function getGenotypeActiveZone(genoma::Genoma)
    identify_deadGenes(genoma)
    [(i + genoma.nInputs, gene) for (i, gene) in enumerate(genoma.genotipo) if genoma.ToEvaluate[i]]
end

function getGenotypeDeadZone(genoma::Genoma)
    identify_deadGenes(genoma)
    [(i + genoma.nInputs, gene) for (i, gene) in enumerate(genoma.genotipo) if !genoma.ToEvaluate[i]]
end

function NAND(a::Int, b::Int)
    return a == 1 && b == 1 ? 0 : 1
end

function NAND_defeituosa(genoma::Genoma, a::Int, b::Int)
    return 0
end

function xor(l)
    n = length(l)
    return (l[n] + l[n-1] == 1) ? 1 : 0
end

function muxNand(l)
    len_l = length(l)
    selector_len = floor(Int, log2(len_l))
    inputs = map(string, l[1:(len_l - selector_len)])
    selector = map(string, l[(len_l - selector_len + 1):len_l])

    selector_bin = join(selector)
    selector_int = parse(Int, selector_bin, base=2)

    return inputs[selector_int + 1]
end

function demuxNand(genoma::Genoma, l)
    len_l = length(l)
    selector_len = ceil(Int, log2(genoma.nOutputs))
    input = l[1]
    selector = map(string, l[2:end])

    selector_bin = join(selector)
    selector_int = parse(Int, selector_bin, base=2)

    println(input, selector)
    println(selector_bin, selector_int)
end

function gpiNand(l)
    count = sum(x -> x == 1, l)
    return (count % 2 == 0) ? "0" : "1"
end

function fullAdderNand(genoma::Genoma, l)
    middle = div(length(l), 2) + 1
    input1 = map(string, l[2:middle])
    input2 = map(string, l[(middle+1):end])
    cIn = string(l[1])

    input1s = join(input1)
    input2s = join(input2)
    cIns = cIn

    sum = string(bin(int(input1s, 2) ⊻ int(input2s, 2) ⊻ int(cIns, 2)))
    cOut = string(bin((int(input1s, 2) & int(input2s, 2)) | 
                      (int(input2s, 2) & int(cIns, 2)) | 
                      (int(input1s, 2) & int(cIns, 2))))
    
    sum_bits = sum * cOut
    sum_bits_string = lpad(sum_bits, genoma.nOutputs, '0')

    return sum_bits_string
end

function NAND(a, b)
    return (a == 1 && b == 1) ? 0 : 1
end

function NAND1(a, b)
    return 1
end

function NAND2(a::Int, b::Int)::Int
    return Int(!Bool(b))
end

function NAND3(a::Int, b::Int)::Int
    return Int(!Bool(a))
end

function NAND4(a, b)
    return 0
end

function NAND_defeitos(genoma::Genoma,a, b)
    value = rand()
    if value > genoma.faultChance
        return NAND(a, b)
    else
        type_of_error = rand(1:100)
        if type_of_error <= 44
            return NAND1(a, b)
        elseif type_of_error <= 66
            return NAND2(a, b)
        elseif type_of_error <= 88
            return NAND3(a, b)
        else
            return NAND4(a, b)
        end
    end
end

function NAND_defeituosa(a, b)
    return NAND4(a, b)
end

function getCartesianProduct(lists)
    return collect(product(lists...))
end

function graphvizGenome(genoma::Genoma, name::String)
    genome = getGenotypeActiveZone(genoma)
    println("Graph visualization needs to be adapted for Julia-specific tools like Graphs.jl.")
end

function mutate(genoma::Genoma)
    # Cria uma cópia do genoma atual para aplicar a mutação
    childGenes = Genoma(numberOfGenes = genoma.numberOfGenes, nInputs = genoma.nInputs, nOutputs = genoma.nOutputs, rateMutation = genoma.rateMutation)
    copyGene(genoma, childGenes)
    
    # Define o número de mutações com base na taxa de mutação
    numberOfMutations = max(trunc(Int,genoma.numberOfGenes * genoma.rateMutation), 1)

    for _ in 1:Int(numberOfMutations)
        # Seleciona um índice para aplicar a mutação
        indexMut = rand(2:genoma.numberOfGenes)  # Evita o índice 1, usa 2:self.numberOfGenes
        # Gera dois valores únicos para substituição
        newGene, alternate = randperm(indexMut + genoma.nInputs - 1)[1:2]

        newGene = string(newGene)
        alternate = string(alternate)

        # Escolhe qual entrada será mutada (0 ou 1)
        whichInput = rand(1:2)
        inputs = split(childGenes.genotipo[indexMut], "-")
        input1, input2 = inputs[1], inputs[2]

        if whichInput == 1
            if newGene == input1
                childGenes.genotipo[indexMut] = "$alternate-$input2"
            else
                childGenes.genotipo[indexMut] = "$newGene-$input2"
            end
        else
            if newGene == input2
                childGenes.genotipo[indexMut] = "$input1-$alternate"
            else
                childGenes.genotipo[indexMut] = "$input1-$newGene"
            end
        end
    end

    return childGenes
end

function calculateFitness(genoma::Genoma, logicFunction)
    identify_deadGenes(genoma)
    fitnessCounter = 0
    l = []
    dic = Dict()

    for i in 1:genoma.nInputs
        push!(l, [0, 1])
    end

    TrueTable = getCartesianProduct(l)

    for i in 1:genoma.possiblesOutputs
        ithTrueTable = TrueTable[i]
        #println(ithTrueTable)
        for input in 1:genoma.nInputs
            #println(ithTrueTable[input])
            dic[string(input)] = ithTrueTable[input]
        end
        #println(dic)
        indexOut = genoma.nInputs+1
        for (position, element) in enumerate(genoma.genotipo)
            if genoma.ToEvaluate[position]
                elements = split(element, "-")
                in1 = elements[1]
                in2 = elements[2]
                out = NAND_defeitos(genoma,dic[in1], dic[in2])
                dic[string(indexOut)] = out
            end
            indexOut += 1
        end
        valueList = [dic[string(m)] for m in 1:genoma.nInputs]
        outDicList = [dic[x] for x in genoma.outputsIndexes]
        outDic = join(outDicList)
        outexact = logicFunction(valueList)

        if outDic == outexact
            fitnessCounter += 1
        end
    end

    genoma.fitness = float(fitnessCounter / genoma.possiblesOutputs)
end

function calculateNoiseFitness(genoma::Genoma)
    noise = round(rand(Uniform(-genoma.Stochasticity, genoma.Stochasticity)), digits=4)
    genoma.noiseFitness = genoma.fitness + noise
end

function calculateFitnessNewFaultInjection(genoma::Genoma, logicFunction, indexFault::Int)
    identify_deadGenes(genoma)
    fitnessCounter = 0
    l = []
    dic = Dict()

    for i in 1:genoma.nInputs
        push!(l, [0, 1])
    end

    TrueTable = getCartesianProduct(l)

    for i in 1:genoma.possiblesOutputs
        ithTrueTable = TrueTable[i]
        for input in 1:genoma.nInputs
            dic[string(input)] = ithTrueTable[input]
        end

        indexOut = genoma.nInputs+1
        for (position, element) in enumerate(genoma.genotipo)
            if genoma.ToEvaluate[position]
                elements = split(element, "-")
                in1 = elements[1]
                in2 = elements[2]
                if position == indexFault
                    out = NAND4(dic[in1], dic[in2])
                else
                    out = NAND(dic[in1], dic[in2])
                end
                dic[string(indexOut)] = out
            end
            indexOut += 1
        end

        valueList = [dic[string(m)] for m in 1:genoma.nInputs]
        outDicList = [dic[x] for x in genoma.outputsIndexes]
        outDic = join(outDicList)
        outexact = logicFunction(valueList)

        if outDic == outexact
            fitnessCounter += 1
        end
    end

    return float(fitnessCounter / genoma.possiblesOutputs)
end

function finalFitnessNewFaultInjection(genoma::Genoma, logicFunction)
    totalFaultFitness = 0
    calculateFitness(genoma,logicFunction)
    TruthFitness = genoma.fitness
    activeGenes = getGenotypeActiveZone(genoma)
    n = length(activeGenes)
    for gene in activeGenes
        totalFaultFitness += calculateFitnessNewFaultInjection(genoma,logicFunction, gene[1] - genoma.nInputs)
    end

    genoma.fitness = (TruthFitness + (totalFaultFitness / n)) / 2
end


finalFitnessNewFaultInjection (generic function with 1 method)

# <b>2 <span style='color:#2ae4f5'>|</span> 9 Ideals Circuits </b>

## Parity Generator (Odd) NANds

### 4 bits PG

In [3]:
PG_collection = Dict()

x = "2-3 2-0 0-0 2-5 5-1 8-6 1-3 9-1 4-2 11-10 4-11 5-0 4-14 10-8 3-9 15-7 14-14 13-17 8-3 11-3 11-0 21-13 3-1 26-25 5-5 25-8 6-13 0-4 8-10 32-16 23-16 15-17 23-26 34-26 19-26 7-37 30-24 23-5 16-12 22-37 31-15 29-27 27-34 26-43 43-9 23-34 3-16 47-0 27-3 5-52 44-45 19-47 16-32 41-14 9-54 39-38 31-44 23-10 50-42 59-55"
lx = split(x, " ")
PG_collection["4PG1"] = lx

x = "1-2 4-2 1-5 0-0 7-6 8-8 5-6 8-3 3-7 4-10 3-11 2-6 7-9 0-11 6-1 4-10 13-19 18-11 10-10 6-21 17-17 7-13 12-17 6-20 10-0 20-8 19-3 26-20 5-0 9-21 14-8 29-26 22-28 34-18 30-31 6-7 17-35 40-13 14-39 30-28 38-11 40-3 16-29 36-28 40-2 42-17 12-43 15-8 22-9 23-9 49-6 0-44 3-47 13-35 38-26 58-20 42-47 34-29 11-10 59-41"
lx = split(x, " ")
PG_collection["4PG2"] = lx

x = "2-0 2-3 3-5 5-6 1-0 2-5 6-9 2-10 0-8 8-9 1-8 14-12 5-10 5-6 2-12 17-9 13-16 6-15 17-7 22-7 16-20 11-18 22-7 11-12 13-3 18-13 24-15 5-28 21-16 7-8 6-14 10-5 0-18 21-32 11-11 3-28 8-2 2-11 11-5 16-5 29-39 13-6 29-22 4-37 29-11 4-48 24-44 10-2 51-35 32-30 8-46 22-0 53-23 42-35 31-1 4-32 42-39 5-11 48-38 53-26"
lx = split(x, " ")
PG_collection["4PG3"] = lx

x = "2-0 2-0 3-3 1-6 7-1 2-5 3-8 5-9 8-1 6-10 1-6 12-10 1-7 10-5 3-14 4-7 0-5 16-19 11-17 20-3 3-23 8-16 0-9 25-12 8-6 9-20 15-29 24-18 27-18 4-27 29-29 15-30 11-34 12-33 36-14 27-25 23-33 25-25 2-39 30-29 6-19 12-10 23-19 29-39 14-20 13-32 15-28 2-9 12-14 43-17 41-47 36-42 22-29 28-26 40-2 57-2 3-10 5-60 22-14 43-35"
lx = split(x, " ")
PG_collection["4PG4"] = lx

x = "1-3 3-1 1-5 3-3 1-2 2-6 2-4 2-2 6-8 1-10 11-5 5-3 0-11 15-15 7-6 1-6 8-1 2-15 18-5 1-3 6-15 13-7 15-22 11-21 24-27 25-21 2-15 0-22 26-0 25-4 22-31 4-29 24-30 34-32 23-2 0-3 23-10 28-8 41-28 39-5 17-29 4-11 28-37 32-7 13-20 44-43 39-46 50-32 42-41 44-13 3-5 7-11 10-20 56-51 19-0 39-53 2-37 1-28 11-46 60-62"
lx = split(x, " ")
PG_collection["4PG5"] = lx

x = "3-0 0-1 5-1 4-6 5-0 4-2 6-7 5-3 7-2 11-8 5-10 3-9 6-3 3-0 17-3 7-12 12-7 6-9 11-3 18-8 2-12 24-20 17-22 9-23 25-23 13-25 13-0 10-16 21-29 29-25 21-28 20-8 15-30 29-2 20-15 30-11 30-31 38-4 34-41 1-34 38-25 13-1 45-19 23-20 27-25 7-4 6-15 29-37 34-11 1-19 43-37 29-13 35-5 5-36 35-30 53-15 59-48 13-44 37-20 55-33"
lx = split(x, " ")
PG_collection["4PG6"] = lx

x = "2-0 4-4 0-3 6-0 6-3 8-7 2-9 10-9 1-1 12-10 4-5 6-12 3-8 2-10 14-15 8-14 11-17 12-12 1-4 15-20 23-4 2-16 7-17 23-21 16-23 6-28 9-26 3-10 27-25 32-32 22-31 34-18 33-5 1-10 27-26 12-20 2-8 2-29 10-40 39-0 17-4 23-19 14-19 0-24 11-4 6-48 43-4 12-47 47-7 17-9 27-39 30-5 10-39 47-33 9-11 52-2 53-3 5-21 48-30 39-27"
lx = split(x, " ")
PG_collection["4PG7"] = lx

x = "2-2 0-3 5-0 4-6 1-2 1-4 3-5 10-6 3-4 2-4 12-2 12-4 2-8 9-16 10-17 18-2 13-17 1-20 19-17 13-1 2-23 1-0 17-0 22-22 11-27 7-27 13-18 9-4 7-7 30-22 26-32 25-27 31-12 21-6 31-2 15-21 1-28 4-10 11-20 6-30 39-27 5-12 1-11 16-13 18-6 43-23 27-24 20-46 32-2 47-6 37-45 44-10 5-43 41-44 56-13 26-1 8-29 44-54 55-34 42-43"
lx = split(x, " ")
PG_collection["4PG8"] = lx

x = "0-1 1-4 3-1 6-3 0-0 4-0 1-1 9-5 4-8 3-11 10-6 7-5 8-9 2-5 1-4 13-3 13-12 18-1 11-13 0-10 22-19 15-16 3-23 2-24 22-14 22-21 14-29 29-6 27-2 8-16 33-6 28-6 23-3 5-21 7-37 30-30 39-5 9-39 30-10 27-2 15-32 44-11 22-41 33-31 6-31 24-29 14-25 29-6 35-10 23-41 9-36 12-29 27-24 47-16 8-42 47-24 16-16 2-41 29-58 56-32"
lx = split(x, " ")
PG_collection["4PG9"] = lx

60-element Vector{SubString{String}}:
 "0-1"
 "1-4"
 "3-1"
 "6-3"
 "0-0"
 "4-0"
 "1-1"
 "9-5"
 "4-8"
 "3-11"
 ⋮
 "12-29"
 "27-24"
 "47-16"
 "8-42"
 "47-24"
 "16-16"
 "2-41"
 "29-58"
 "56-32"

## Multiplex Nands

### 2x1 Mux

In [4]:
MUX_collection = Dict()

x = "2-0 0-1 3-3 1-1 2-1 3-2 2-1 6-1 9-8 2-11 2-7 0-6 2-12 11-3 10-15 17-3 16-13 10-13 3-1 14-9 1-22 0-17 20-7 22-23 5-19 27-8 7-7 2-0 4-26 30-9 8-6 24-0 14-16 22-25 32-5 28-5 25-18 31-23 16-3 32-39 14-1 1-11 6-0 42-17 5-12 12-41 11-34 23-14 18-30 7-9 49-38 2-50 18-6 46-17 37-31 21-28 5-53 46-8 24-18 7-34"
lx = split(x, " ")
MUX_collection["2x1MUX1"] = lx

x = "0-1 2-3 1-1 2-5 4-0 0-2 1-7 3-6 5-5 10-3 2-6 9-13 6-9 4-6 3-10 8-15 2-1 6-6 2-4 10-14 19-13 1-6 11-0 12-25 4-4 13-27 1-24 9-1 29-16 11-7 30-14 0-5 19-16 13-10 19-10 22-1 5-0 32-9 8-38 0-2 10-41 31-37 11-10 21-43 27-33 10-23 40-43 20-0 4-50 44-38 30-16 3-17 45-51 19-46 21-47 32-12 1-51 2-43 41-45 13-7"
lx = split(x, " ")
MUX_collection["2x1MUX2"] = lx

x = "2-0 0-3 1-3 4-2 3-0 0-1 4-4 6-9 6-6 11-10 9-1 6-7 14-2 15-14 16-13 3-2 8-18 6-16 15-12 19-12 7-21 16-18 23-3 10-7 22-12 10-14 15-22 1-17 27-8 3-20 4-11 1-2 16-13 13-22 5-23 14-2 28-4 35-31 5-21 6-15 2-39 29-28 5-25 10-36 2-32 21-24 20-45 3-7 17-42 20-5 52-43 45-43 40-44 22-12 0-25 1-24 25-9 1-54 6-51 4-34"
lx = split(x, " ")
MUX_collection["2x1MUX3"] = lx

x = "1-1 0-1 2-3 5-2 2-3 0-7 5-6 5-5 3-8 4-2 11-0 12-5 8-4 15-3 1-14 9-5 18-16 5-2 9-0 21-8 15-17 22-11 10-24 1-8 18-5 2-14 7-0 10-20 18-17 25-29 21-19 20-30 17-15 4-29 17-17 0-9 23-2 5-28 32-31 9-27 31-1 29-20 13-18 39-20 23-7 12-0 4-13 32-20 40-50 43-1 3-41 11-1 32-46 24-0 54-5 43-12 26-47 41-45 19-48 29-43"
lx = split(x, " ")
MUX_collection["2x1MUX4"] = lx

x = "0-0 2-2 2-1 0-5 6-6 3-4 0-4 2-0 1-10 5-4 3-12 3-2 6-2 9-5 13-16 9-6 17-4 9-13 12-6 8-0 16-22 22-19 12-4 17-9 11-17 8-10 25-0 2-18 5-23 16-21 0-17 27-16 23-7 25-2 2-0 3-33 33-37 21-9 38-34 38-33 25-35 15-29 44-6 32-10 28-8 4-25 6-27 49-40 4-8 12-36 29-23 17-32 53-4 54-14 17-20 18-7 21-57 6-38 17-37 9-5"
lx = split(x, " ")
MUX_collection["2x1MUX5"] = lx

x = "2-1 2-1 2-2 5-3 0-2 1-4 0-8 7-1 2-5 3-10 0-1 5-7 10-6 8-7 4-10 5-13 3-3 12-9 7-10 21-3 18-7 4-1 16-19 23-3 4-22 7-15 9-21 2-9 0-16 28-14 24-14 5-0 28-25 8-32 12-13 8-34 15-20 0-33 11-28 0-1 3-6 26-26 30-24 7-14 33-40 20-3 23-48 7-46 5-2 37-46 19-45 41-45 11-50 19-27 1-44 35-18 7-22 29-10 26-2 34-56"
lx = split(x, " ")
MUX_collection["2x1MUX6"] = lx

x = "0-2 2-1 4-4 2-0 5-5 4-7 1-4 6-0 0-2 10-11 1-10 7-12 7-4 6-5 0-2 0-15 17-0 0-5 19-11 1-13 15-0 6-17 21-2 16-4 2-9 3-23 26-26 4-10 10-5 15-7 22-30 25-10 28-21 31-3 15-11 4-16 15-23 5-24 14-7 11-34 4-36 42-31 28-4 45-30 27-37 25-33 41-16 32-2 45-10 29-24 19-29 49-33 41-40 45-10 54-35 44-42 28-31 44-0 7-19 31-10"
lx = split(x, " ")
MUX_collection["2x1MUX7"] = lx

x = "1-1 3-2 3-2 5-3 3-5 0-4 0-3 0-3 10-6 7-2 7-3 12-7 7-4 11-10 3-3 9-8 3-3 17-0 0-7 21-6 1-19 8-7 9-17 7-8 7-20 20-18 27-8 2-10 13-17 4-10 6-21 6-25 6-3 13-16 12-5 5-14 16-6 1-8 31-35 31-0 36-5 18-34 7-34 17-2 46-9 3-30 34-5 4-16 4-41 3-18 16-33 44-22 6-32 38-40 10-45 26-34 14-57 25-53 58-40 44-46"
lx = split(x, " ")
MUX_collection["2x1MUX8"] = lx

x = "0-0 0-0 4-4 2-2 2-1 3-0 1-4 0-6 8-3 4-8 5-6 2-9 6-2 0-14 4-14 6-2 6-11 12-4 4-13 3-14 9-10 22-10 11-13 9-3 19-14 17-27 17-6 7-3 25-14 1-18 17-16 20-20 11-23 6-6 24-22 9-32 22-18 33-24 2-19 35-15 39-3 13-40 6-11 32-34 32-15 44-33 10-21 21-22 2-7 27-38 9-21 10-11 9-18 16-20 52-3 7-56 3-1 40-5 36-6 13-7"
lx = split(x, " ")
MUX_collection["2x1MUX9"] = lx

60-element Vector{SubString{String}}:
 "0-0"
 "0-0"
 "4-4"
 "2-2"
 "2-1"
 "3-0"
 "1-4"
 "0-6"
 "8-3"
 "4-8"
 ⋮
 "10-11"
 "9-18"
 "16-20"
 "52-3"
 "7-56"
 "3-1"
 "40-5"
 "36-6"
 "13-7"

### 4x2 Mux

In [5]:
x = "4-3 6-4 3-5 1-7 9-5 2-4 5-2 5-7 4-12 14-0 15-1 15-8 13-11 15-10 12-1 6-19 8-1 0-8 14-6 9-22 24-8 10-17 19-15 15-9 22-16 17-16 1-1 15-27 15-1 11-1 19-30 6-32 16-0 29-6 33-15 40-22 4-38 3-22 18-23 36-24 29-4 22-16 24-35 17-34 10-10 44-44 18-12 24-20 53-50 16-42 43-52 20-23 38-46 55-46 52-33 3-24 54-18 19-39 15-5 62-27"
lx = split(x, " ")
MUX_collection["4x2MUX1"] = lx

x = "1-0 0-1 4-4 5-5 2-9 0-8 11-9 7-8 10-12 5-8 13-13 0-15 1-5 9-14 5-13 13-15 1-7 6-18 7-22 12-3 11-15 10-23 25-10 17-8 3-1 12-9 19-0 7-32 14-12 6-28 35-28 18-17 14-17 28-29 37-3 37-26 20-16 28-27 32-40 41-26 2-32 12-39 41-32 5-28 7-4 20-39 42-26 1-22 7-7 54-48 55-47 42-34 0-30 44-23 34-46 7-60 55-27 44-3 54-25 39-41"
lx = split(x, " ")
MUX_collection["4x2MUX2"] = lx

x = "2-1 4-2 2-4 5-3 7-0 7-10 6-10 6-11 3-4 4-7 14-5 9-1 4-12 9-8 17-0 9-4 20-12 1-5 16-19 0-16 7-1 2-18 17-22 23-25 17-23 29-6 9-1 30-1 17-20 21-7 34-17 15-21 19-26 14-7 6-5 0-21 29-21 14-7 8-14 36-8 21-20 29-33 10-25 11-48 35-36 6-23 45-43 16-20 40-53 2-33 0-0 9-37 36-5 12-43 23-34 23-54 9-40 50-13 59-60 24-42"
lx = split(x, " ")
MUX_collection["4x2MUX3"] = lx

x = "5-3 5-1 5-5 6-6 0-8 4-2 4-5 10-6 11-7 5-0 15-14 12-14 12-14 11-13 14-10 4-6 14-3 0-17 5-9 3-8 9-20 22-21 16-22 0-19 18-15 17-23 26-30 7-30 15-5 19-16 26-23 14-24 14-9 24-37 22-6 10-12 7-13 15-9 26-14 39-35 0-0 36-21 36-32 32-19 0-39 21-41 16-21 9-19 6-13 52-30 29-54 45-17 0-35 4-17 9-0 49-7 44-27 42-48 43-23 51-18"
lx = split(x, " ")
MUX_collection["4x2MUX4"] = lx

x = "5-1 3-5 1-6 7-4 8-5 1-5 10-8 6-1 2-10 9-12 4-14 1-15 0-9 14-18 7-3 19-16 5-6 5-6 7-23 22-23 5-22 18-6 27-10 15-21 28-4 14-8 17-12 26-31 13-19 16-12 22-1 36-24 30-28 15-25 26-32 26-23 5-28 41-15 25-35 41-27 22-17 26-30 19-11 27-30 2-45 47-29 34-6 7-35 2-12 30-0 4-54 4-55 8-51 44-55 48-44 32-44 19-5 4-26 53-44 51-28"
lx = split(x, " ")
MUX_collection["4x2MUX5"] = lx

x = "0-0 4-3 0-7 7-5 4-4 4-9 9-11 12-12 2-13 4-11 1-10 16-11 12-14 8-8 10-17 12-17 5-12 2-20 22-23 13-8 6-4 9-25 8-24 17-24 0-9 18-28 9-24 11-32 20-5 19-27 27-14 29-8 29-16 5-5 4-32 20-20 15-3 37-33 10-8 18-37 8-16 30-26 44-21 32-16 5-7 1-15 51-27 47-50 31-42 12-5 50-10 5-41 37-11 33-51 30-22 5-36 16-34 27-17 46-6 45-29"
lx = split(x, " ")
MUX_collection["4x2MUX6"] = lx

x = "0-2 4-2 4-7 7-5 5-4 10-10 10-9 7-6 11-3 1-5 3-1 13-4 15-17 2-15 9-16 9-0 2-10 2-20 13-14 7-15 1-20 24-4 4-9 16-19 10-14 29-11 21-25 3-28 12-7 25-25 35-21 4-22 36-37 21-14 10-37 39-16 6-35 17-25 33-39 36-8 14-29 26-38 41-19 28-39 19-46 44-44 20-12 42-5 48-10 22-44 16-10 43-15 37-57 54-10 2-53 34-48 58-40 5-15 37-54 14-38"
lx = split(x, " ")
MUX_collection["4x2MUX7"] = lx

x = "3-5 4-6 2-4 1-8 9-6 10-3 1-7 6-0 1-3 1-3 5-12 4-8 9-4 2-13 8-18 5-2 5-16 16-11 10-8 0-22 18-13 22-8 17-16 2-22 7-18 24-22 0-16 18-27 28-30 5-5 33-29 30-6 24-7 28-3 26-8 25-25 13-0 29-24 42-3 42-4 45-1 28-6 16-26 20-15 22-15 11-9 48-43 50-13 8-52 20-46 48-52 6-2 40-52 26-14 26-47 54-36 10-45 9-52 63-63 22-60"
lx = split(x, " ")
MUX_collection["4x2MUX8"] = lx

x = "0-2 5-1 3-5 7-5 0-0 10-7 8-5 5-5 8-13 11-9 0-1 1-4 12-13 5-17 16-8 12-4 2-1 6-10 4-8 6-24 3-24 15-21 22-22 1-0 19-8 15-0 6-28 10-9 3-27 28-11 34-0 9-30 24-27 37-21 3-27 27-37 21-26 12-14 8-3 13-37 39-2 46-19 17-3 32-22 10-9 40-5 40-24 24-43 13-42 16-18 40-13 43-12 10-52 21-0 56-54 58-35 30-28 36-29 52-59 38-46"
lx = split(x, " ")
MUX_collection["4x2MUX9"] = lx

60-element Vector{SubString{String}}:
 "0-2"
 "5-1"
 "3-5"
 "7-5"
 "0-0"
 "10-7"
 "8-5"
 "5-5"
 "8-13"
 "11-9"
 ⋮
 "43-12"
 "10-52"
 "21-0"
 "56-54"
 "58-35"
 "30-28"
 "36-29"
 "52-59"
 "38-46"

# <b>3 <span style='color:#2ae4f5'>|</span> Genetic Algorith Class </b>


In [6]:
using Dates
using Plots
using Printf

mutable struct GeneticAlgorithm
    y::Int
    maxGeneration::Int
    startTime::DateTime
    data_atual::DateTime
    timeTaken::Period
    totalGeneration::Int
    countGeneration::Int
    gotMaxFitness::Bool
    keepOptmization::Bool
    keepOptmizationIterations::Int
    DisplayEnabler::Bool
    fitnessList::Vector{String}
end

function GeneticAlgorithm(;y=10, maxGeneration=4000000)
    return GeneticAlgorithm(
        y, maxGeneration, now(), today(), Dates.Millisecond(0), 0, 0, false, false, 10000, true, String[]
    )
end

function setLambda!(ga::GeneticAlgorithm, newLambda::Int)
    ga.y = newLambda
end

function setKeepOptmization!(ga::GeneticAlgorithm, keepOptmizationIterations::Int)
    ga.keepOptmization = true
    ga.keepOptmizationIterations = keepOptmizationIterations
end

function getTotalGeneration(ga::GeneticAlgorithm)
    return ga.totalGeneration
end

function getTimeTaken(ga::GeneticAlgorithm)
    return ga.timeTaken
end

function incGeneration!(ga::GeneticAlgorithm)
    ga.totalGeneration = ga.totalGeneration + 1
end

function disableDisplay!(ga::GeneticAlgorithm)
    ga.DisplayEnabler = false
end

function display(ga::GeneticAlgorithm, guess::Vector{String}, fitness, noiseFitness, totalGeneration)
    sguess = join(guess, " ")
    timeDiff = now() - ga.startTime
    println(@sprintf("%s\t %.5f\t %.5f\t Geração: %d Tempo: %s", sguess, fitness, noiseFitness, totalGeneration, timeDiff))
end

function addToFitnessList!(ga::GeneticAlgorithm, childFitness)
    push!(ga.fitnessList, string(childFitness))
    sort!(ga.fitnessList)  # Mantém a lista ordenada
end

function showBarPlot(ga::GeneticAlgorithm, genome_id::String, samplingLen::Int)
    arr = ga.fitnessList
    elements_count = Dict{String, Int}()
    yNormalization = samplingLen / 10.0

    for element in arr
        elements_count[element] = get(elements_count, element, 0) + 1
    end

    for (key, value) in elements_count
        println("$key: $value")
    end

    values = collect(keys(elements_count))
    frequency = [elements_count[key] / yNormalization for key in values]

    bar(values, frequency, label="F2", color=:plum)
    plot!(values, frequency, marker=:o, label="Out", color=:purple)

    xlabel!("Fitness", fontsize=24)
    ylabel!("Frequency ($(Int(yNormalization)))", fontsize=24)
    grid!(:y)
    savefig(genome_id * ".eps")
    display(plot())
end

function getBestGenomeWithSize(listChild)
    bestChild = listChild[1]
    for child in listChild
        if child.noiseFitness > bestChild.noiseFitness ||
           (child.noiseFitness == bestChild.noiseFitness && length(child.indexActiveGenes) ≤ length(bestChild.indexActiveGenes))
            bestChild = child
        end
    end
    return bestChild
end

function getBestGenome(listChild)
    bestChild = listChild[1]
    for child in listChild
        if child.fitness > bestChild.fitness
            bestChild = child
        end
    end
    return bestChild
end

function evolution(ga::GeneticAlgorithm, genome, logicFunction)
    bestParent = genome
    if isempty(bestParent.genotipo)
        generate_parent(bestParent)
    end

    finalFitnessNewFaultInjection(bestParent,logicFunction)

    if ga.DisplayEnabler
        display(ga, bestParent.genotipo, bestParent.fitness, bestParent.noiseFitness, ga.totalGeneration)
    end

    listGenomes = []

    while true
        incGeneration!(ga)
        empty!(listGenomes)
        finalFitnessNewFaultInjection(bestParent,logicFunction)
        push!(listGenomes, bestParent)

        for i in 1:ga.y
            child = Genoma(numberOfGenes = bestParent.numberOfGenes,nInputs = bestParent.nInputs,nOutputs = bestParent.nOutputs, rateMutation = bestParent.rateMutation)
            copyGene(mutate(bestParent),child)
            finalFitnessNewFaultInjection(child,logicFunction)
            push!(listGenomes, child)
        end

        copyGene(getBestGenome(listGenomes),bestParent)
        addToFitnessList!(ga, bestParent.fitness)
        if ga.totalGeneration % 1000 == 0 && ga.DisplayEnabler
            display(ga, bestParent.genotipo, bestParent.fitness, bestParent.noiseFitness, ga.totalGeneration)
        end

        if ga.totalGeneration >= ga.maxGeneration
            break
        end

        if bestParent.fitness >= 1
            ga.gotMaxFitness = true
            if ga.DisplayEnabler
                display(ga, bestParent.genotipo, bestParent.fitness, bestParent.noiseFitness, ga.totalGeneration)
                break
            end
        end
    end

    ga.timeTaken = now() - ga.startTime
    if ga.DisplayEnabler
        println("The end in: $(ga.timeTaken)")
    end

    return bestParent
end


evolution (generic function with 1 method)

# <b>5 <span style='color:#2ae4f5'>|</span> Evolution </b>

In [6]:
nGenes = 200
nOutputs = 1
nInputs = 4
rateMutation = 0.5


genoma = Genoma(numberOfGenes=nGenes, nInputs=nInputs, nOutputs=nOutputs, rateMutation=rateMutation)

geneticAlgorithm = GeneticAlgorithm(y = 10, maxGeneration=400000)

evolution(geneticAlgorithm,genoma,gpiNand)


1-4 2-5 5-1 3-7 6-3 4-7 5-9 4-8 2-5 7-9 6-6 12-7 3-9 3-14 16-15 13-3 14-6 11-16 20-18 21-17 14-16 10-24 13-1 1-25 19-26 3-27 10-14 2-1 20-18 4-32 27-29 17-32 14-19 30-33 31-11 23-35 25-17 16-25 33-9 26-1 3-32 20-27 33-30 31-47 4-37 40-3 8-28 16-22 3-8 15-26 16-4 15-25 7-8 14-15 23-56 15-31 10-7 25-7 6-19 7-6 42-26 4-44 28-33 61-6 19-22 33-18 69-11 26-39 24-1 45-24 44-25 40-49 74-22 40-40 32-4 77-25 2-70 46-15 3-3 82-56 71-41 36-41 16-28 77-65 17-12 19-46 3-11 3-6 51-4 58-14 1-36 29-94 66-4 58-47 53-39 15-23 6-74 14-38 102-55 83-71 69-22 76-66 65-101 26-27 15-100 48-45 62-48 78-16 8-75 7-9 90-51 80-57 2-96 97-115 34-100 79-45 35-96 34-73 91-53 47-67 104-70 67-35 101-75 115-55 10-65 15-50 109-27 20-84 118-89 47-19 8-88 107-127 45-46 110-90 8-25 8-1 65-75 46-71 142-46 143-17 44-40 66-12 58-64 113-47 83-17 59-131 85-105 116-53 5-125 139-91 148-100 154-74 18-148 82-141 12-25 71-18 27-37 155-119 145-156 43-59 30-140 160-35 128-53 123-78 167-27 50-70 165-91 7-108 71-54 144-105 151-3 106-78 50

InterruptException: InterruptException:

# <b>5 <span style='color:#2ae4f5'>|</span> Teste </b>

In [ ]:
nGenes = 200
nOutputs = 1
nInputs = 4
rateMutation = 0.5

genoma = Genoma(numberOfGenes=nGenes, nInputs=nInputs, nOutputs=nOutputs, rateMutation=rateMutation)
genoma.setGenotipo(PG_collection["4PG9"])


MethodError: MethodError: no method matching setGenotipo(::Genoma, ::Vector{SubString{String}})
The function `setGenotipo` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  setGenotipo(::Genoma, !Matched::Vector{String})
   @ Main ~/Documents/UFPE/Logic Circuits Evolution/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W1sZmlsZQ==.jl:60
  setGenotipo(::Genoma, !Matched::String)
   @ Main ~/Documents/UFPE/Logic Circuits Evolution/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W1sZmlsZQ==.jl:42
